In [1]:
from src.data_preprocessor import DataProcessor
from src.cross_validation import CrossValidation
from src.evaluation import Evaluation
from models.knn import KNN
from models.null_model import NullModelClassification, NullModelRegression
from data_configs.configs import *
import statistics
import numpy as np

config = machine_config
data_processor = DataProcessor(config=config)
cross_validator = CrossValidation(config=config)
classification_nullmodel = NullModelClassification(config=config)
regression_nullmodel = NullModelRegression(config=config)
knn_model = KNN(config)
null_model = NullModelRegression(config=config)

### Data Load and Preprocessing ###

In [2]:
raw_data = data_processor.load_data()

raw_data_2 = raw_data.drop(columns=['vendor_name', 'model_name', 'ERP'])

data_1 = data_processor.impute_missing_values(raw_data_2)

data_2 = data_processor.encode_nominal_features(data_1)

data_3 = data_processor.encode_ordinal_features(data_2)

## KNN Model ##

### Hyperparameter Tuning ###

In [3]:
data_train, data_val = cross_validator.random_partition(data_3, random_state=42)

In [4]:
features=['MYCT', 'MMIN', 'MMAX', 'CACH', 'CHMIN', 'CHMAX']

In [5]:
gamma = 1/(statistics.stdev(data_train[config['target_column']]))
gamma

0.0071906613411797165

#### Tuning k ####

In [6]:
hyperparameters = np.arange(1,15,1)
scores_dict = {}

for k in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
        
        data_train_standardized = data_processor.standardize_data(train_set_1, train_set_1, features=features)
        data_val_standardized = data_processor.standardize_data(train_set_1,data_val,features=features)  

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=k, gamma=gamma)['Predicted Value']
        score_1 = Evaluation().mean_squared_error(data_val_standardized[config['target_column']], predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with k={k}: {average_score}")
    scores_dict[k] = average_score

best_k = min(scores_dict, key=scores_dict.get)
print(f"Best k is {best_k} with the lowest average MSE score of {scores_dict[best_k]}")


Average score with k=1: 18692.819047619043
Average score with k=2: 17661.420809542098
Average score with k=3: 18995.569669901513
Average score with k=4: 19611.49917804948
Average score with k=5: 20429.088741870484
Average score with k=6: 21261.43363838865
Average score with k=7: 21813.683190323467
Average score with k=8: 22263.245673862275
Average score with k=9: 23109.50510701653
Average score with k=10: 24044.911264984657
Average score with k=11: 24791.148339346157
Average score with k=12: 25524.400740610767
Average score with k=13: 26372.642408441294
Average score with k=14: 27191.311096247235
Best k is 2 with the lowest average score of 17661.420809542098


#### Tuning Gamma ####

In [7]:
hyperparameters = np.arange(0.4,1.6,0.2)
scores_dict = {}

for g in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
        
        data_train_standardized = data_processor.standardize_data(train_set_1, train_set_1, features=features)
        data_val_standardized = data_processor.standardize_data(train_set_1,data_val,features=features)  

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=best_k, gamma=gamma*g)['Predicted Value']
        score_1 = Evaluation().mean_squared_error(data_val_standardized[config['target_column']], predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with g={round(g,2)}: {average_score}")
    scores_dict[g] = average_score

best_g = min(scores_dict, key=scores_dict.get)
print(f"Best g is {round(best_g,2)} with the lowest average MSE score of {scores_dict[best_g]}")


Average score with g=0.4: 17774.013336162112
Average score with g=0.6: 17734.10464995688
Average score with g=0.8: 17696.562832538002
Average score with g=1.0: 17661.420809542098
Average score with g=1.2: 17628.695996424074
Average score with g=1.4: 17598.390491571034
Average score with g=1.6: 17570.491502459376
Best g is 1.6 with the lowest average score of 17570.491502459376


### Model Performance ###

In [8]:
mse_scores = []
mae_scores = []
r2_scores = []
pearson_scores = []
null_model_scores = []

for i, (train_set, test_set) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
    
    data_train_standardized = data_processor.standardize_data(train_set, train_set, features=features)
    data_test_standardized = data_processor.standardize_data(train_set,test_set,features=features)  

    # Train and evaluate 
    predictions_1 = knn_model.knn_regression(data_test_standardized, data_train_standardized, k=best_k, gamma=best_g*gamma)['Predicted Value']
    
    mse_score = Evaluation.mean_squared_error(data_test_standardized[config['target_column']], predictions_1)
    mae_score = Evaluation.mean_absolute_error(data_test_standardized[config['target_column']], predictions_1)
    r2_score = Evaluation.r2_coefficient(data_test_standardized[config['target_column']], predictions_1)
    pearson_score = Evaluation.pearsons_correlation(data_test_standardized[config['target_column']], predictions_1)
    
    mse_scores.append(mse_score)
    mae_scores.append(mae_score)
    r2_scores.append(r2_score)
    pearson_scores.append(pearson_score)

    null_model_prediction = null_model.naive_regression(test_set)
    null_model_score = Evaluation.mean_squared_error(test_set[config['target_column']],null_model_prediction)
    null_model_scores.append(null_model_score) 


average_mse_score = sum(mse_scores) / len(mse_scores)
average_mae_score = sum(mae_scores) / len(mae_scores)
average_r2_score = sum(r2_scores) / len(r2_scores)
average_pearson_score = sum(pearson_scores) / len(pearson_scores)
average_null_model_score = sum(null_model_scores) / len(null_model_scores)
study_score = Evaluation.mean_squared_error(raw_data['PRP'],raw_data['ERP'])
print(f"Previous study MSE score (ERP vs PRP): {study_score}")
print(f"Average null model MSE score: {average_null_model_score}")
print(f"Average MSE score for k={best_k}, g={round(best_g,2)}: {average_mse_score}")
print(f"Average MAE score for k={best_k}, g={round(best_g,2)}: {average_mae_score}")
print(f"Average r2 score for k={best_k}, g={round(best_g,2)}: {average_r2_score}")
print(f"Average pearson score for k={best_k}, g={round(best_g,2)}: {average_pearson_score}")


Previous study MSE score (ERP vs PRP): 1737.3349282296651
Average null model MSE score: 19073.865381296106
Average MSE score for k=2, g=1.6: 7512.193936429954
Average MAE score for k=2, g=1.6: 34.97747282263274
Average r2 score for k=2, g=1.6: 0.6523904298418604
Average pearson score for k=2, g=1.6: 0.8250440267334402


## Edited KNN ##

### Hyperparameter Tuning ###

In [9]:
epsilon = statistics.stdev(data_3[config['target_column']])
epsilon

160.83073308779512

#### Tuning Epsilon ####

In [10]:
hyperparameters = np.arange(0.1,2,0.2)
scores_dict = {}

for e in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
        
        edited_train_set = knn_model.edited_knn_regression(train_set_1,epsilon=epsilon*e)

        data_train_standardized = data_processor.standardize_data(edited_train_set, edited_train_set, features=features)
        data_val_standardized = data_processor.standardize_data(edited_train_set,data_val,features=features)  

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=best_k, gamma=gamma*best_g)['Predicted Value']
        score_1 = Evaluation().mean_squared_error(data_val_standardized[config['target_column']], predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average score with e={round(e,2)}: {average_score}")
    scores_dict[e] = average_score

best_e = min(scores_dict, key=scores_dict.get)
print(f"Best e is {round(best_e,2)} with the lowest average MSE score of {scores_dict[best_e]}")


Average score with e=0.1: 38778.07607077246
Average score with e=0.3: 30175.29096799274
Average score with e=0.5: 28840.71963897106
Average score with e=0.7: 30513.332061414454
Average score with e=0.9: 27420.17226756271
Average score with e=1.1: 26313.910440406624
Average score with e=1.3: 24855.84380880785
Average score with e=1.5: 24464.997863820077
Average score with e=1.7: 22967.24611659685
Average score with e=1.9: 22948.6021755046
Best e is 1.9 with the lowest average score of 22948.6021755046


#### Tuning k ####

In [11]:
hyperparameters = np.arange(1,10,1)
scores_dict = {}

for k in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
        
        edited_train_set = knn_model.edited_knn_regression(train_set_1,epsilon=epsilon*best_e)

        data_train_standardized = data_processor.standardize_data(edited_train_set, edited_train_set, features=features)
        data_val_standardized = data_processor.standardize_data(edited_train_set,data_val,features=features)  

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=k, gamma=gamma*best_g)['Predicted Value']
        score_1 = Evaluation().mean_squared_error(data_val_standardized[config['target_column']], predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average score with k={k}: {average_score}")
    scores_dict[k] = average_score

best_k = min(scores_dict, key=scores_dict.get)
print(f"Best k is {best_k} with the lowest average MSE score of {scores_dict[best_k]}")


Average score with k=1: 24474.57619047619
Average score with k=2: 22948.6021755046
Average score with k=3: 23417.646879133114
Average score with k=4: 23214.49718321472
Average score with k=5: 23913.237952320113
Average score with k=6: 24213.783169836
Average score with k=7: 24775.255090475788
Average score with k=8: 25360.162611618678
Average score with k=9: 25766.91935639654
Best k is 2 with the lowest average score of 22948.6021755046


#### Tuning gamma ####

In [12]:
hyperparameters = np.arange(0.4,1.6,0.2)
scores_dict = {}

for g in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
        
        edited_train_set = knn_model.edited_knn_regression(train_set_1,epsilon=epsilon*best_e)

        data_train_standardized = data_processor.standardize_data(edited_train_set, edited_train_set, features=features)
        data_val_standardized = data_processor.standardize_data(edited_train_set,data_val,features=features)  

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=best_k, gamma=g*gamma)['Predicted Value']
        score_1 = Evaluation().mean_squared_error(data_val_standardized[config['target_column']], predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with g={round(g,2)}: {average_score}")
    scores_dict[g] = average_score

best_g = min(scores_dict, key=scores_dict.get)
print(f"Best g is {round(best_g,2)} with the lowest average MSE score of {scores_dict[best_g]}")


Average score with g=0.4: 23044.22232910942
Average score with g=0.6: 23026.51392144658
Average score with g=0.8: 23009.47285176583
Average score with g=1.0: 22993.13512696233
Average score with g=1.2: 22977.530115185957
Average score with g=1.4: 22962.68046607314
Average score with g=1.6: 22948.6021755046
Best g is 1.6 with the lowest average score of 22948.6021755046


In [13]:
mse_scores = []
mae_scores = []
r2_scores = []
pearson_scores = []
null_model_scores = []

for i, (train_set, test_set) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
    
    edited_train_set = knn_model.edited_knn_regression(train_set,epsilon=epsilon*best_e)

    data_train_standardized = data_processor.standardize_data(edited_train_set, edited_train_set, features=features)
    data_test_standardized = data_processor.standardize_data(edited_train_set,test_set,features=features)  

    predictions_1 = knn_model.knn_regression(data_test_standardized, data_train_standardized, k=best_k, gamma=best_g*gamma)['Predicted Value']

    mse_score = Evaluation.mean_squared_error(data_test_standardized[config['target_column']], predictions_1)
    mae_score = Evaluation.mean_absolute_error(data_test_standardized[config['target_column']], predictions_1)
    r2_score = Evaluation.r2_coefficient(data_test_standardized[config['target_column']], predictions_1)
    pearson_score = Evaluation.pearsons_correlation(data_test_standardized[config['target_column']], predictions_1)

    mse_scores.append(mse_score)
    mae_scores.append(mae_score)
    r2_scores.append(r2_score)
    pearson_scores.append(pearson_score)
    
    null_model_prediction = null_model.naive_regression(test_set)
    null_model_score = Evaluation.mean_squared_error(test_set[config['target_column']],null_model_prediction)
    null_model_scores.append(null_model_score) 

average_mse_score = sum(mse_scores) / len(mse_scores)
average_mae_score = sum(mae_scores) / len(mae_scores)
average_r2_score = sum(r2_scores) / len(r2_scores)
average_pearson_score = sum(pearson_scores) / len(pearson_scores)
average_null_model_score = sum(null_model_scores) / len(null_model_scores)
study_score = Evaluation.mean_squared_error(raw_data['PRP'],raw_data['ERP'])
print(f"Previous study MSE score (ERP vs PRP): {study_score}")
print(f"Average null model MSE score: {average_null_model_score}")
print(f"Average MSE score for k={best_k}, g={round(best_g,2)}: {average_mse_score}")
print(f"Average MAE score for k={best_k}, g={round(best_g,2)}: {average_mae_score}")
print(f"Average r2 score for k={best_k}, g={round(best_g,2)}: {average_r2_score}")
print(f"Average pearson score for k={best_k}, g={round(best_g,2)}: {average_pearson_score}")

Previous study MSE score (ERP vs PRP): 1737.3349282296651
Average null model MSE score: 19073.865381296106
Average MSE score for k=2, g=1.6: 7127.359746567536
Average MAE score for k=2, g=1.6: 34.31842830852905
Average r2 score for k=2, g=1.6: 0.6771279508406438
Average pearson score for k=2, g=1.6: 0.8389989397901749


## Condensed Knn ##

#### Tuning Epsilon ####

In [14]:
hyperparameters = np.arange(0.1,1.5,0.1)
scores_dict = {}

for e in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
        
        condesed_train_set = knn_model.condensed_knn_regression(train_set_1,epsilon=epsilon*e)

        data_train_standardized = data_processor.standardize_data(condesed_train_set, condesed_train_set, features=features)
        data_val_standardized = data_processor.standardize_data(condesed_train_set,data_val,features=features)

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=best_k, gamma=gamma*best_g)['Predicted Value']
        score_1 = Evaluation().mean_squared_error(data_val_standardized[config['target_column']], predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with e={round(e,2)}: {average_score}")
    scores_dict[e] = average_score

best_e = min(scores_dict, key=scores_dict.get)
print(f"Best e is {round(best_e,2)} with the lowest average MSE score of {scores_dict[best_e]}")


Average score with e=0.1: 16831.909325167486
Average score with e=0.2: 16622.10938873105
Average score with e=0.3: 16676.972152913604
Average score with e=0.4: 16597.17988102914
Average score with e=0.5: 17271.058594167604
Average score with e=0.6: 17124.081283354746
Average score with e=0.7: 17799.686305741547
Average score with e=0.8: 17648.96196470669
Average score with e=0.9: 18461.582560444607
Average score with e=1.0: 18377.353322048326
Average score with e=1.1: 21654.81205508003
Average score with e=1.2: 22051.750319240764
Average score with e=1.3: 21005.169735518288
Average score with e=1.4: 22102.608294654634
Best e is 0.4 with the lowest average score of 16597.17988102914


#### Tuning k ####

In [15]:
hyperparameters = np.arange(1,10,1)
scores_dict = {}

for k in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
        
        condesed_train_set = knn_model.condensed_knn_regression(train_set_1,epsilon=epsilon*best_e)

        data_train_standardized = data_processor.standardize_data(condesed_train_set, condesed_train_set, features=features)
        data_val_standardized = data_processor.standardize_data(condesed_train_set,data_val,features=features)

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=k, gamma=gamma*best_g)['Predicted Value']
        score_1 = Evaluation().mean_squared_error(data_val_standardized[config['target_column']], predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with k={k}: {average_score}")
    scores_dict[k] = average_score

best_k = min(scores_dict, key=scores_dict.get)
print(f"Best k is {best_k} with the lowest average MSE score of {scores_dict[best_k]}")


Average score with k=1: 17034.11904761905
Average score with k=2: 16550.627740105392
Average score with k=3: 17074.35149938288
Average score with k=4: 17889.902595161308
Average score with k=5: 18579.63829344037
Average score with k=6: 20596.358101661066
Average score with k=7: 21998.601298828304
Average score with k=8: 23560.51594537509
Average score with k=9: 24401.306788125607
Best k is 2 with the lowest average score of 16550.627740105392


#### Tuning Gamma ####

In [16]:
hyperparameters = np.arange(0.4,1.6,0.2)
scores_dict = {}

for g in hyperparameters: 
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
        
        condesed_train_set = knn_model.condensed_knn_regression(train_set_1,epsilon=epsilon*best_e)

        data_train_standardized = data_processor.standardize_data(condesed_train_set, condesed_train_set, features=features)
        data_val_standardized = data_processor.standardize_data(condesed_train_set,data_val,features=features)  

        predictions_1 = knn_model.knn_regression(data_val_standardized, data_train_standardized, k=best_k, gamma=g*gamma)['Predicted Value']
        score_1 = Evaluation().mean_squared_error(data_val_standardized[config['target_column']], predictions_1)
        scores.append(score_1)

    average_score = sum(scores) / len(scores)
    print(f"Average MSE score with g={round(g,2)}: {average_score}")
    scores_dict[g] = average_score

best_g = min(scores_dict, key=scores_dict.get)
print(f"Best g is {round(best_g,2)} with the lowest average MSE score of {scores_dict[best_g]}")


Average score with g=0.4: 16766.557055963276
Average score with g=0.6: 17206.52999071452
Average score with g=0.8: 16927.210128364364
Average score with g=1.0: 16732.402670162737
Average score with g=1.2: 16914.488121853763
Average score with g=1.4: 16754.375624588796
Average score with g=1.6: 16735.741788337487
Best g is 1.0 with the lowest average score of 16732.402670162737


### Model Performance ###

In [17]:
mse_scores = []
mae_scores = []
r2_scores = []
pearson_scores = []
null_model_scores = []

for i, (train_set, test_set) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
    
    condesed_train_set_train_set = knn_model.condensed_knn_regression(train_set,epsilon=epsilon*best_e)

    data_train_standardized = data_processor.standardize_data(condesed_train_set_train_set, condesed_train_set_train_set, features=features)
    data_test_standardized = data_processor.standardize_data(condesed_train_set_train_set,test_set,features=features)  

    predictions_1 = knn_model.knn_regression(data_test_standardized, data_train_standardized, k=best_k, gamma=best_g*gamma)['Predicted Value']

    mse_score = Evaluation.mean_squared_error(data_test_standardized[config['target_column']], predictions_1)
    mae_score = Evaluation.mean_absolute_error(data_test_standardized[config['target_column']], predictions_1)
    r2_score = Evaluation.r2_coefficient(data_test_standardized[config['target_column']], predictions_1)
    pearson_score = Evaluation.pearsons_correlation(data_test_standardized[config['target_column']], predictions_1)

    mse_scores.append(mse_score)
    mae_scores.append(mae_score)
    r2_scores.append(r2_score)
    pearson_scores.append(pearson_score)
    
    null_model_prediction = null_model.naive_regression(test_set)
    null_model_score = Evaluation.mean_squared_error(test_set[config['target_column']],null_model_prediction)
    null_model_scores.append(null_model_score) 

average_mse_score = sum(mse_scores) / len(mse_scores)
average_mae_score = sum(mae_scores) / len(mae_scores)
average_r2_score = sum(r2_scores) / len(r2_scores)
average_pearson_score = sum(pearson_scores) / len(pearson_scores)
average_null_model_score = sum(null_model_scores) / len(null_model_scores)
study_score = Evaluation.mean_squared_error(raw_data['PRP'],raw_data['ERP'])
print(f"Previous study MSE score (ERP vs PRP): {study_score}")
print(f"Average null model MSE score: {average_null_model_score}")
print(f"Average MSE score for k={best_k}, g={round(best_g,2)}: {average_mse_score}")
print(f"Average MAE score for k={best_k}, g={round(best_g,2)}: {average_mae_score}")
print(f"Average r2 score for k={best_k}, g={round(best_g,2)}: {average_r2_score}")
print(f"Average pearson score for k={best_k}, g={round(best_g,2)}: {average_pearson_score}")

Previous study MSE score (ERP vs PRP): 1737.3349282296651
Average null model MSE score: 19073.865381296106
Average MSE score for k=2, g=1.0: 7988.312280238902
Average MAE score for k=2, g=1.0: 39.19266728667226
Average r2 score for k=2, g=1.0: 0.5946295516054472
Average pearson score for k=2, g=1.0: 0.8051534168319087
